# Retrival Based two Agent: (LLaMA 3.1 (8B)) 1. REASONING AGENT + 2. RETRIEVER AGENT(RAG)

🟠 LLaMA 3.1 (Local) — model = "llama3.1:8b"

Retriever Agent runs fully offline with vector DB.

Reasoning Agent is LLaMA 3.1 via Ollama.

No API key, no cloud dependency.

Slightly slower but privacy-preserving.

Best for PrivateRAG and regulated environments.

In [1]:
# =========================================================
# TWO-AGENT LOCAL RAG — LLaMA 3.1 (8B) via OLLAMA
# =========================================================

from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
import chromadb
import ollama

# =========================================================
# AGENT 2: LLaMA 3.1 (8B) — REASONING AGENT
# =========================================================
class LlamaAgent:
    def answer(self, context, question):
        prompt = f"""
You are a document-grounded AI assistant.

Rules:
- Answer ONLY using the provided context.
- If the answer is not in the context, say exactly:
  Not found in the document.

Context:
{context}

Question:
{question}
"""
        response = ollama.chat(
            model="llama3.1:8b",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return response["message"]["content"]


# =========================================================
# AGENT 1: RETRIEVER AGENT
# =========================================================
class RetrieverAgent:
    def __init__(self, pdf_path):
        self.embedder = SentenceTransformer("all-MiniLM-L6-v2")
        self.client = chromadb.Client()
        self.collection = self.client.get_or_create_collection(
            name="llama31-8b-docs"
        )
        self._ingest_pdf(pdf_path)

    def _ingest_pdf(self, pdf_path):
        reader = PdfReader(pdf_path)
        text = "\n".join(page.extract_text() for page in reader.pages)

        words = text.split()
        chunks = [
            " ".join(words[i:i + 300])
            for i in range(0, len(words), 300)
        ]

        for i, chunk in enumerate(chunks):
            self.collection.add(
                documents=[chunk],
                ids=[str(i)],
                embeddings=[self.embedder.encode(chunk).tolist()]
            )

    def retrieve(self, question, k=3):
        q_embedding = self.embedder.encode(question).tolist()
        results = self.collection.query(
            query_embeddings=[q_embedding],
            n_results=k
        )
        return "\n".join(results["documents"][0])


# =========================================================
# ORCHESTRATOR
# =========================================================
retriever_agent = RetrieverAgent("Ai_Test_Book.pdf")
llama_agent = LlamaAgent()

# =========================================================
# LIMIT TO ONLY 3 QUESTIONS
# =========================================================
MAX_QUESTIONS = 3
question_count = 0

questions = [
    "What is Artificial Intelligence?",
    "What is your salary?",
    "Explain agent-based AI systems."
]

# =========================================================
# RUN TWO-AGENT LOCAL RAG
# =========================================================
for question in questions:
    if question_count >= MAX_QUESTIONS:
        print("❌ Question limit reached. Session ended.")
        break

    print("\n--------------------------------")
    print(f"Question: {question}")

    # AGENT 1 → RETRIEVE
    context = retriever_agent.retrieve(question)

    # AGENT 2 → REASON
    answer = llama_agent.answer(context, question)

    print(f"Answer: {answer}")

    question_count += 1



--------------------------------
Question: What is Artificial Intelligence?
Answer: Artificial Intelligence (AI) is a branch of computer science that focuses on building systems capable of performing tasks that normally require human intelligence. These tasks include reasoning, learning, problem-solving, perception, and language understanding. The term Artificial Intelligence was first introduced by John McCarthy in 1956 during the Dartmouth Conference.

--------------------------------
Question: What is your salary?
Answer: Not found in the document.

--------------------------------
Question: Explain agent-based AI systems.
Answer: According to the context (Page 9), Agent-based systems consist of autonomous components called agents. Each agent has:

- A goal
- Memory
- Decision logic

Multi-agent systems improve reliability and scalability.
